# Floating Point Assignment

To run this notebook, use the provided file `floating_point_assignment.ipynb` and place it into your working directory.

```bash
source venv/bin/activate
jupyter lab --ip 0.0.0.0
```

Then, follow the link from the standard out of the command. This contains a token to provide you access. To turn in your work, save this notebook and provide the file.

# Definitions (25)

We will start with a definition of floating point and double using IEEE format definitions. They are

* 32-bit: 1 sign bit, 8 exp bits, 23 frac bits
* 64-bit: 1 sign bit, 11 exp bits, 52 frac bits
* 8-bit: 1 sign bit, 4 exp bits, 3 frac bits

1. Provide the representation of the value 3.14159265 in 32-bit IEEE format.
1. Provide the representation of the value 3.14159265 in 64-bit IEEE.
1. Provide the representation of the value 3.14159265 in the 8 bit format
1. Compare the precision loss of each value in comparison to 64-bit.
1. Choose a repeating number of your choosing (ie 1/3) and show the precision loss comparing the 8-bit format to 32-bit.

For numbers 1-3, provide your value in hex as that is easier. The above values should be completed by hand. You may use the techniques in part two to check your work.

# Precision (25)

In [ ]:
import numpy as np
from pymap3d.ecef import geodetic2ecef

In this section, you will use latitude, longitude, altitude coordinates to perform some calculations. The first is a coordinate change into Earth Centered, Earth Fixed (ECEF). You will then use a method for calculating the euclidean distance. You will then make observations on the precision of your calculations using three different floating point definitions. I have chosen UCCS and UC Boulder as examples. Replace at least one of these locations with those that you choose.

## Points (5 points each)

1. Replace location coordinates
2. Perform calculations with each of `np.float64`, `np.float32`, and `np.float16`
3. Brief discussion of observations

In [ ]:
help(geodetic2ecef)

In [ ]:
help(np.linalg.norm)

### 64-bit

In [ ]:
uccs_64 = np.array([38.8936117,-104.8005516, 1965.96])
ucb_64 = np.array([40.0073943,-105.2662901, 1661.16])

In [ ]:
uccs_64_ecef = np.array([*geodetic2ecef(*uccs_64)])
uccs_64_ecef

In [ ]:
ucb_64_ecef = np.array([*geodetic2ecef(*ucb_64)])
ucb_64_ecef

In [ ]:
np.linalg.norm(uccs_64_ecef - ucb_64_ecef)

### 32-bit

In [ ]:
uccs_32 = uccs_64.astype(np.float32)
ucb_32 = ucb_64.astype(np.float32)

In [ ]:
uccs_32_ecef = np.array([*geodetic2ecef(*uccs_32)])
uccs_32_ecef

In [ ]:
ucb_32_ecef = np.array([*geodetic2ecef(*ucb_32)])
ucb_32_ecef

In [ ]:
np.linalg.norm(uccs_32_ecef - ucb_32_ecef)

### 16-bit

In [ ]:
# Complete on your own

### Discussion

Provide a brief discussion here

# Number Line (10)

On a number line, place a sequence of at least 10 numbers (both +/-), along with the largest possible value (+/-) represented in the 8-bit tiny notation. You may choose the numbers yourself. You should observe the distance between the numbers, and make some comments.  Here is an example of two numbers:

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
x = np.array([-10.5, -0.5, 0.5, 10.5])
y = np.zeros_like(x)

In [ ]:
plt.plot(x, y);
plt.plot(x, y, 'x');

### Discussion

Provide a brief discussion here